<a href="https://colab.research.google.com/github/harshinikoppula30/agent/blob/main/My_langgraph2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU google_generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.2 MB/s eta 0:00:00


In [ ]:
import os
import getpass

# LangGraph imports
from langgraph.graph import StateGraph, END

# LangChain imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


In [ ]:
os.environ['GOOGLE_API_KEY']=getpass.getpass("enter the api key")

enter the api key··········


In [ ]:
 llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0.2
)

In [ ]:
def get_symptom(state:dict)->dict:
  symptom=input("enter a symptom")
  state["symptom"]=symptom
  return state


In [ ]:
def classify_symptom(state: dict) -> dict:
    prompt = (
        "You are a helpful Medical Assistant. Classify the symptoms below into one of the categories:\n"
        "- General\n"
        "- Emergency\n"
        "- Mental Health\n"
        f"Symptom: {state['symptom']}\n"
        "Respond only with one word: General, Emergency, or Mental Health.\n"
        "#Example: input: I have fever, Output: General"
    )

    response = llm.invoke([HumanMessage(content=prompt)])
    category = response.content.strip()

    print(f"LLM classifies the symptom as: {category}")  # debug

    state["category"] = category
    return state


In [ ]:
def symptom_router(state: dict) -> dict:
    cat = state["category"].lower()
    if "general" in cat:
        return "general"
    elif "emergency" in cat:
        return "emergency"
    elif "mental" in cat:
        return "mental_health"
    else:
        return "general"


In [ ]:
def general_node(state: dict) -> dict:
    state["answer"] = f"{state['symptom']} : seems general..."
    return state

def emergency_node(state: dict) -> dict:
    state["answer"] = f"{state['symptom']} : It is a Medical Emergency..."
    return state

def mental_health_node(state: dict) -> dict:
    state["answer"] = f"{state['symptom']} : seems like a mental health issue..."
    return state


In [ ]:
builder = StateGraph(dict)

builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental_health", mental_health_node)

builder.add_edge("get_symptom", "classify")

builder.add_conditional_edges("classify", symptom_router, {
    "general": "general",
    "emergency": "emergency",
    "mental_health": "mental_health"
})

builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("mental_health", END)


In [ ]:
graph = builder.compile()
final_state = graph.invoke({})
print("final Output\n")
print(final_state["answer"])


enter a symptomheadache
LLM classifies the symptom as: General
final Output

headache : seems general...
